In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
caminho_xlsx = r'C:\Users\Robert\Documents\Python\base_de_dados\relatorio.xlsx'

In [7]:


try:
    df_bruto = pd.read_excel(caminho_xlsx, header=None, engine='openpyxl')
    df_bruto.drop(df_bruto.index[:10], inplace=True)
    df_bruto.dropna(how='all', inplace=True)
    df_bruto.reset_index(drop=True, inplace=True)
    
    # Vamos usar um método mais seguro que não depende da posição exata.
    # Encontramos o índice da primeira ocorrência de "Tipo" e começamos a partir daí.
    primeiro_tipo_idx = df_bruto[df_bruto[0] == 'Tipo'].index[0]
    df_bruto = df_bruto.iloc[primeiro_tipo_idx:].copy()
    df_bruto.reset_index(drop=True, inplace=True)

    print("Arquivo lido e preparado.")
    print("DataFrame pronto para o loop:")

except Exception as e:
    print(f"Ocorreu um erro na leitura: {e}")
    df_bruto = pd.DataFrame()

df_bruto.head(5)

Arquivo lido e preparado.
DataFrame pronto para o loop:


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,Tipo,NaN,NaN,NaN,NaN,Nome,NaN,NaN,NaN,NaN,Grupo,NaN,NaN,NaN,NaN,Nome
1,1,NaN,NaN,NaN,NaN,PRODUTO ACABADO,NaN,NaN,NaN,NaN,10,NaN,NaN,NaN,NaN,RACAO BOVINO
2,Item,Situação,Descrição,Complemento,UM,Origem,IPI,Cl. Fiscal,Data Cadastro,Data Inativo,NaN,NaN,NaN,NaN,NaN,NaN
3,109041,NaN,BOVINUCLEO BEZERROS 30KG,NaN,SC,C,0,23099090,2017-07-24 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Tipo,NaN,NaN,NaN,NaN,Nome,NaN,NaN,NaN,NaN,Grupo,NaN,NaN,NaN,NaN,Nome


In [8]:

# --- 2. IMPLEMENTAÇÃO FINAL COM BUSCA ROBUSTA ---

memoria_categoria = {
    'Tipo': None, 'Nome_Tipo': None,
    'Grupo': None, 'Nome_Grupo': None
}
dados_finais = []

if not df_bruto.empty:
    for index, row in df_bruto.iterrows():
        try:
            # --- REGRA 1: É A LINHA DE CABEÇALHO "Tipo"? ---
            if str(row.iloc[0]).strip() == 'Tipo':
                
                if index + 1 < len(df_bruto):
                    linha_de_valores = df_bruto.iloc[index + 1]
                    
                    # Captura os Nomes, que são estáveis
                    nome_tipo_atual = linha_de_valores.iloc[5]
                    nome_grupo_atual = linha_de_valores.iloc[15]
                    
                    # --- LÓGICA DE BUSCA PELO VIZINHO ---
                    tipo_atual = None
                    # Procura o valor do Tipo à esquerda do Nome do Tipo
                    for i in range(4, -1, -1): # Itera das colunas 4 a 0
                        val = pd.to_numeric(linha_de_valores.iloc[i], errors='coerce')
                        if pd.notna(val):
                            tipo_atual = val
                            break # Encontrou, pode parar de procurar

                    grupo_atual = None
                    # Procura o valor do Grupo à esquerda do Nome do Grupo
                    for i in range(14, 5, -1): # Itera das colunas 14 a 6
                        val = pd.to_numeric(linha_de_valores.iloc[i], errors='coerce')
                        if pd.notna(val):
                            grupo_atual = val
                            break # Encontrou, pode parar de procurar
                    
                    # Atualiza a memória com tudo que foi encontrado
                    memoria_categoria['Tipo'] = tipo_atual
                    memoria_categoria['Nome_Tipo'] = nome_tipo_atual
                    memoria_categoria['Grupo'] = grupo_atual
                    memoria_categoria['Nome_Grupo'] = nome_grupo_atual
                    
                    print(f"--- MEMÓRIA ATUALIZADA: Tipo {tipo_atual}, Grupo {grupo_atual} ---")
                continue

            # --- REGRA 2: É UMA LINHA DE DADOS DE ITEM? ---
            item_id = pd.to_numeric(row.iloc[0], errors='coerce')
            
            if pd.notna(item_id) and pd.notna(row.iloc[2]):
                dados_finais.append({
                    'Item': int(item_id),
                    'Descrição': str(row.iloc[2]),
                    'Tipo': memoria_categoria['Tipo'],
                    'Nome_Tipo': memoria_categoria['Nome_Tipo'],
                    'Grupo': memoria_categoria['Grupo'],
                    'Nome_Grupo': memoria_categoria['Nome_Grupo']
                })
        except (IndexError, TypeError, ValueError):
            continue

# --- 3. CRIAÇÃO DO DATAFRAME FINAL ---
df_final = pd.DataFrame(dados_finais)
if not df_final.empty:
    for col in ['Item', 'Tipo', 'Grupo']:
        df_final[col] = pd.to_numeric(df_final[col], errors='coerce').astype('Int64')

# --- EXIBIÇÃO DO RESULTADO ---
print("\n--- DataFrame Final Estruturado ---")
display(df_final)

--- MEMÓRIA ATUALIZADA: Tipo 1, Grupo 10 ---
--- MEMÓRIA ATUALIZADA: Tipo 1, Grupo 26 ---
--- MEMÓRIA ATUALIZADA: Tipo 1, Grupo 41 ---
--- MEMÓRIA ATUALIZADA: Tipo 1, Grupo 87 ---
--- MEMÓRIA ATUALIZADA: Tipo 1, Grupo 996 ---
--- MEMÓRIA ATUALIZADA: Tipo 1, Grupo 998 ---
--- MEMÓRIA ATUALIZADA: Tipo 1, Grupo 999 ---
--- MEMÓRIA ATUALIZADA: Tipo 2, Grupo 18 ---
--- MEMÓRIA ATUALIZADA: Tipo 2, Grupo 70 ---
--- MEMÓRIA ATUALIZADA: Tipo 3, Grupo 47 ---
--- MEMÓRIA ATUALIZADA: Tipo 4, Grupo 86 ---
--- MEMÓRIA ATUALIZADA: Tipo 4, Grupo 87 ---
--- MEMÓRIA ATUALIZADA: Tipo 5, Grupo 1 ---
--- MEMÓRIA ATUALIZADA: Tipo 5, Grupo 2 ---
--- MEMÓRIA ATUALIZADA: Tipo 5, Grupo 6 ---
--- MEMÓRIA ATUALIZADA: Tipo 5, Grupo 8 ---
--- MEMÓRIA ATUALIZADA: Tipo 5, Grupo 9 ---
--- MEMÓRIA ATUALIZADA: Tipo 5, Grupo 11 ---
--- MEMÓRIA ATUALIZADA: Tipo 5, Grupo 12 ---
--- MEMÓRIA ATUALIZADA: Tipo 5, Grupo 13 ---
--- MEMÓRIA ATUALIZADA: Tipo 5, Grupo 15 ---
--- MEMÓRIA ATUALIZADA: Tipo 5, Grupo 16 ---
--- MEMÓRIA 

,Item,Descrição,Tipo,Nome_Tipo,Grupo,Nome_Grupo
0,109041,BOVINUCLEO BEZERROS 30KG,1,PRODUTO ACABADO,10,RACAO BOVINO
1,79000,FARINHA DE VISCERAS DE FRANGO,1,PRODUTO ACABADO,26,SUB PRODUTO
2,79003,RESÍDUOS DE PENA,1,PRODUTO ACABADO,26,SUB PRODUTO
3,79004,RESÍDUOS DE SANGUE,1,PRODUTO ACABADO,26,SUB PRODUTO
4,193,BOBINA COXA E SOBRECOXA RESFRIADA 530 MM X 350...,1,PRODUTO ACABADO,41,EMBALAGEM PRIMARIA
...,...,...,...,...,...,...
17460,120325,PRESTACAO SERVICO DE INSPECAO DO DIGESTOR,25,SERVICOS,74,SERVICOS DE INSPECAO/CALIBRACAO
17461,120579,MODULO CEP CONTROLE ESTATISTICO DE PROCESSOS S...,25,SERVICOS,75,SERVICO CONTRATACAO DE SOFTWARE FABRICA DE RACAO
17462,120578,MODULO DE ESTOQUE SO AUTOMACAO,25,SERVICOS,75,SERVICO CONTRATACAO DE SOFTWARE FABRICA DE RACAO
17463,120577,MODULO DE RASTREABILIDADE SO AUTOMACAO,25,SERVICOS,75,SERVICO CONTRATACAO DE SOFTWARE FABRICA DE RACAO


In [9]:
caminho_salvamento = r'C:\Users\Robert\Documents\Python\base_de_dados'
caminho_final = os.path.join(caminho_salvamento, 'd_produtos_almoxarifado.xlsx')

df_final.to_excel(caminho_final, index=False)
